# partlist-io2md

A tool to create a Markdown part list from a BOM (Bill Of Materials) extracted
from a project designed with [Stud.IO CAD](https://www.bricklink.com/v3/studio/download.page).
For each submodel in the project a dedicated list is created inside the markdown file.

> MIT License
> 
> Copyright &copy; 2022 by Alessandro Varesi
> 
> Permission is hereby granted, free of charge, to any person obtaining a copy
> of this software and associated documentation files (the "Software"), to deal
> in the Software without restriction, including without limitation the rights
> to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
> copies of the Software, and to permit persons to whom the Software is
> furnished to do so, subject to the following conditions:
>
> The above copyright notice and this permission notice shall be included in all
> copies or substantial portions of the Software.
>
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
> IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
> FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
> AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
> LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
> OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
> SOFTWARE.
>

In [232]:
%%capture

import sys
# Install pip packages in current Jupyter kernel
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install urllib
!{sys.executable} -m pip install beautifulsoup4

## Choose the IO file name

In next code cell set the CSV filename (with path if needed)

In [233]:
iofile = "../../sets/10281/10281-bags.io"

Now `Run all` cells of this notebook

In [245]:
import zipfile
import pandas as pd
from IPython.display import display, Markdown

with zipfile.ZipFile(iofile) as z:
    # print (z.infolist())

    with z.open('.info', pwd = b'soho0909') as f:
        pass
        # print (f.readlines())

    model_file = z.open('model.ldr',  pwd = b'soho0909')
    
    for l in model_file.readlines():
      print (l.strip().decode('utf-8-sig'))

    model_file.seek(0)

0 FILE 10281-bags
0 Untitled Model
0 Name:  10281-bags
0 Author:
0 CustomBrick
1 16 90.000000 0.000000 -380.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 1
1 16 480.000000 0.000000 -320.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 2
1 16 770.000000 0.000000 -90.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 3
1 16 1020.000000 0.000000 -170.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 4
1 16 110.000000 0.000000 -1100.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 5
1 16 740.000000 4.000000 -1120.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 6
1 16 1120.000000 4.000000 -1000.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 unbagged
0 NOFILE
0 FILE Bag 1
0 Bag 1
0 Name:  Bag 1
0 

In [235]:
# Get color information from StudioColorDefinition.txt 
# ... this file is from Stud.io application
import math

colors = pd.read_csv("StudioColorDefinition.txt", sep='\t')
# colors

def getColorInfo(color: int):
    """Get 'BL Color Code' and 'BL Color Name'

    ### Parameter
    color : int
        the LDraw color code

    ### Returns
    bl_color : dictionary
        BL Color Code : int
            the Bricklink color code (integer)
        BL Color Name : str 
            the Bricklinr color name (str)
    
    """

    # (colors['BL Color Code'] == colors['BL Color Code']) is atrick to check is not a NaN
    r = colors.loc[(colors['LDraw Color Code'] == color) & (colors['BL Color Code'] == colors['BL Color Code'])]

    return {
        'BL Color Code': int(r.iloc[0]['BL Color Code']),
        'BL Color Name': r.iloc[0]['BL Color Name']
    }

# colors = colors[['LDraw Color Code','BL Color Code','BL Color Name']]

print(getColorInfo(84))
print(getColorInfo(28))

{'BL Color Code': 150, 'BL Color Name': 'Medium Nougat'}
{'BL Color Code': 69, 'BL Color Name': 'Dark Tan'}


In [236]:
# Get various information from Bricklink site by getting info direct from html pages
# Not using Official API that need a AccessKey given only to registered sellers users.
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup


part_url = 'https://www.bricklink.com/v2/catalog/catalogitem.page?P={part}&idColor={blcolor}'

def getPartInfo(part : str, blcolor: int):
    req = Request(f'https://www.bricklink.com/v2/catalog/catalogitem.page?P={part}&idColor={blcolor}', headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req, timeout=10).read()
    soup = BeautifulSoup(html, 'html.parser')
    # Extract data from html
    description = soup.find('span', {'id': 'item-name-title'}).string
    weight = soup.find('span', {'id': 'item-weight-info'}).string
    dimension = soup.find('span', {'id': 'dimSec'}).string
    image = f'https://img.bricklink.com/ItemImage/PN/{blcolor}/{part}.png'
    return { 
        'partname': part, 
        'description': description, 
        'weight': weight,
        'dimension': dimension,
        'image': image
        }

print(getPartInfo('3036', 69))

{'partname': '3036', 'description': 'Plate 6 x 8', 'weight': '6.4g', 'dimension': '6 x 8 in studs', 'image': 'https://img.bricklink.com/ItemImage/PN/69/3036.png'}


In [237]:
# Scan model file and fill the BOM dictionary structure
# part = { 
# }
submodels = {}
parts = {}

class Model:
    def __init__(self, name: str):
        self.name = name
        self.BOM = {}
        self.submodels = {}

    def addSubmodel(self, sm_name: str):
        sm_id = sm_name.lower()
        if sm_id not in submodels:
            print (f"{self.name}.addSubmodel({sm_name})")
            submodels[sm_id] = Model(sm_name)
        self.submodels[sm_id] = submodels[sm_id]

    def addPart(self, part: str, color: int):
        part_id = f'{color}_{part}'
        #print (f"{self.name}.addPart({part_id}) ",end='') #DEBUG
        if part_id not in parts:
        #    print (f"LegoPart({part}, {color}) ",end='')
            parts[part_id] = LegoPart(part, color)
        if part_id in self.BOM:
            self.BOM[part_id]['qty'] += 1
        else:
            self.BOM[part_id]={'name': part, 'color': color, 'qty': 1}
        print(f'{self.BOM}')

    def printBOM(self, isSubModel = False):
        if not isSubModel:
            print(f'MODEL : {self.name}')
            print('--------------------')
        else:
            print(f'    {self.name}')
            print('    ----------')
        print(f" BOM: {self.BOM}")
        for id, p in self.BOM.items():
            print (f'{p.name}({parts[id]["colorname"]}): {p.qty} ')
        for sm in self.submodels.values():
            sm.printBOM(isSubModel = True)

class LegoPart:
    def __init__(self, partname: str, color: int):
        self.partname = partname.split('.')[0]
        self.color = color
        c = getColorInfo(color)
        self.blcolor = c['BL Color Code'] 
        self.colorname = c['BL Color Name']
    
    def getPartInfo(self):
        p = getPartInfo(self.partname, self.blcolor)
        self.description = p['description']
        self.weight = p['weight']
        self.dimension = p['dimension']
        self.image = p['image']


l = model_file.readline().strip().decode('utf-8-sig').split(' ')

model = Model(' '.join(l[2:]))
# throw away next 4 lines
model_file.readline()
model_file.readline()
model_file.readline()

print (f'Creating BOM for model : {model.name}')

thisModel = model

for l in model_file.readlines():

    line = l.strip().decode('utf-8-sig').split(' ')
    # check for line type 
    if line[0] == '0':
        if line[1] == 'FILE':
            sm_name = " ".join(line[2:])
            sm_id = sm_name.lower()
            if sm_id not in submodels:
                submodels[sm_id] = Model(sm_name)
            # if model is created from a line "1" the name is lowercase
            # rewrite name with the one defined on FILE 
            submodels[sm_id] = sm_name # rewrite name 
            thisModel = submodels[sm_id]
            #print (f'Create submodel : {thisModel.name}')
    elif line[0] == '1':
        color = int(line[1]) # color
        part = " ".join(line[14:]) # part
        if color == 16:    # this is a submodel in the BOM
            thisModel.addSubmodel(part)
        elif color == 24:
            pass
        else:
            thisModel.addPart(part, color)

for p in parts.values():
    print(f'{p.partname} - Color: {p.color} ({p.colorname})')
    
    

Creating BOM for model : 10281-bags
10281-bags.addSubmodel(bag 1)
10281-bags.addSubmodel(bag 2)
10281-bags.addSubmodel(bag 3)
10281-bags.addSubmodel(bag 4)
10281-bags.addSubmodel(bag 5)
10281-bags.addSubmodel(bag 6)
10281-bags.addSubmodel(unbagged)
{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 1}}
{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 2}}
{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 2}, '0_2445.dat': {'name': '2445.dat', 'color': 0, 'qty': 1}}
{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 2}, '0_2445.dat': {'name': '2445.dat', 'color': 0, 'qty': 2}}
{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 2}, '0_2445.dat': {'name': '2445.dat', 'color': 0, 'qty': 3}}
{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 2}, '0_2445.dat': {'name': '2445.dat', 'color': 0, 'qty': 4}}
{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 2}, '0_2445.dat': {'name': '2445.dat', 'color': 0, 'qty': 4}, '72_4477.dat': {'name': '4477.dat'

In [244]:
submodels['Bag 1'].BOM


{'28_3036.dat': {'name': '3036.dat', 'color': 28, 'qty': 2},
 '0_2445.dat': {'name': '2445.dat', 'color': 0, 'qty': 4},
 '72_4477.dat': {'name': '4477.dat', 'color': 72, 'qty': 4},
 '72_87609.dat': {'name': '87609.dat', 'color': 72, 'qty': 6},
 '15_61485.dat': {'name': '61485.dat', 'color': 15, 'qty': 1},
 '71_30414.dat': {'name': '30414.dat', 'color': 71, 'qty': 10},
 '0_87079.dat': {'name': '87079.dat', 'color': 0, 'qty': 12},
 '0_15068.dat': {'name': '15068.dat', 'color': 0, 'qty': 24},
 '71_3623.dat': {'name': '3623.dat', 'color': 71, 'qty': 4},
 '0_59895.dat': {'name': '59895.dat', 'color': 0, 'qty': 5},
 '3_96874.dat': {'name': '96874.dat', 'color': 3, 'qty': 1}}

In [239]:
md = "Picture | Qty | Code | Description | Color |\n"
md += "--------|----:|------|-------------|-------|\n"
for i in df.itertuples():
    item = i.BLItemNo
    qty = i.Qty
    description = i.PartName
    color = i.ColorName
    colorid = i.BLColorId
    image = f"![{item}](https://img.bricklink.com/P/{colorid:.0f}/{item}.jpg)"
    link = f"[{item}](https://www.bricklink.com/v2/catalog/catalogitem.page?P={item}&idColor={colorid:.0f})"

    md += f"{image}| {qty:.0f} | {link} | {description} | {color}\n"

display(Markdown(md))

NameError: name 'df' is not defined

In [ ]:
outmd = "## Here is your markdown block\n"
outmd += "Copy next code block and paste in your _md_ file\n"
outmd += "```(copy)\n"
outmd += md
outmd += "```\n"

display(Markdown(outmd))